IMPORTS

In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import ast

BASES

MERGES et BASES

In [2]:
df_title_basics = pd.read_csv('BD/title.basics.tsv.gz', compression = 'gzip', sep = '\t')
df_title_ratings = pd.read_csv('BD/title.ratings.tsv.gz', compression = 'gzip', sep = '\t')

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_14904/3795572048.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_title_basics = pd.read_csv('BD/title.basics.tsv.gz', compression = 'gzip', sep = '\t')


In [3]:
df1 = pd.merge(df_title_basics,
               df_title_ratings,
               how = 'outer',
               on = 'tconst')

In [4]:
df_title_ratings, df_title_basics = 0, 0

In [5]:
df_title_akas = pd.read_csv('BD/title.akas.tsv.gz', compression = 'gzip', sep = '\t')

In [6]:
df_3 = pd.merge(df1,
                df_title_akas,
                how = 'outer',
                left_on = 'tconst',
                right_on = 'titleId')

In [7]:
df1, df_title_akas = 0, 0

In [8]:
df_tmbd_full = pd.read_csv('BD/tmdb_full.csv')

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_14904/1088705580.py:1: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tmbd_full = pd.read_csv('BD/tmdb_full.csv')


In [9]:
df_films = pd.merge(df_tmbd_full,
                df_3,
                how = 'outer',
                left_on = 'imdb_id',
                right_on = 'tconst')

In [10]:
df_tmbd_full, df_3 = 0, 0

CLEAN DU MERGE

In [11]:
# DROP CE QUI N'EST PAS FILM

df_films = df_films[(df_films['titleType'] == 'movie')]

COLONNES NUMERIQUES

In [12]:
# On remplit les cellules vides par 0

df_films[['runtimeMinutes', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']] = df_films[['runtimeMinutes', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']].fillna(0)

In [13]:
# On remplace les '\\N' par 0

df_films[['runtimeMinutes', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']] = df_films[['runtimeMinutes', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']].replace('\\N', 0)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_14904/2320903248.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_films[['runtimeMinutes', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']] = df_films[['runtimeMinutes', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']].replace('\\N', 0)


In [14]:
# On met ces colonnes au format numérique

df_films[['runtimeMinutes', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']] = df_films[['runtimeMinutes', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']].apply(pd.to_numeric)

In [15]:
# On transforme les types de données pour 'release_date'

df_films['release_date'] = pd.to_datetime(df_films['release_date'])
df_films['release_date'] = df_films['release_date'].replace('\\N', 0)
df_films['release_date'] = df_films['release_date'].dt.year

In [16]:
# On transforme les types de données pour 'startYear'

df_films['startYear'] = df_films['startYear'].replace('\\N', 0)
df_films['startYear'] = pd.to_numeric(df_films['startYear'], downcast="integer")

In [17]:
# On crée la colonne year_exact

df_films['year_exact'] = df_films[['release_date', 'startYear']].max(axis = 1)

In [18]:
# On crée la colonne 'Decennie'

df_films['Decennie'] = ''
df_films['year_exact'] = df_films['year_exact'].fillna('0')
df_films['year_exact'] = df_films['year_exact'].replace('nan', '0')
df_films['year_exact'] = df_films['year_exact'].astype(str)

In [19]:
df_films['Decennie'] = df_films['year_exact'].str[:3] + "0's"
df_films['Decennie'] = df_films['Decennie'].replace("0.00's", None)
df_films['year_exact'] = pd.to_numeric(df_films['year_exact'])

RUNTIME

In [20]:
# On met de côté les valeurs nulles

df_null1 = df_films[((df_films['runtimeMinutes'].isna()) & (df_films['runtime'].isna()))]
df_null2 = df_films[((df_films['runtimeMinutes'] == 0) & (df_films['runtime'] == 0))]
df_null3 = df_films[((df_films['runtimeMinutes'] == 0) & (df_films['runtime'].isna()))]
df_null4 = df_films[((df_films['runtimeMinutes'].isna()) & (df_films['runtime'] == 0))]

In [21]:
df_null = pd.concat([df_null1, df_null2, df_null3, df_null4])

In [22]:
df_null1, df_null2, df_null3, df_null4 = 0, 0, 0, 0

In [23]:
# On prend les valeurs utiles

df_utile = df_films[~((df_films['runtimeMinutes'].isna()) & (df_films['runtime'].isna()))]
df_utile = df_utile[~((df_utile['runtimeMinutes'] == 0) & (df_utile['runtime'] == 0))]
df_utile = df_utile[~((df_utile['runtimeMinutes'] == 0) & (df_utile['runtime'].isna()))]
df_utile = df_utile[~((df_utile['runtimeMinutes'].isna()) & (df_utile['runtime'] == 0))]

In [24]:
# Calcul différence de runtime des 2 tables

df_utile['difference_runtime'] = abs(df_utile['runtime'] - df_utile['runtimeMinutes'])

In [25]:
# On met de côté là où les 2 valeurs sont identiques

df_a_garder_1 = df_utile[(df_utile['runtimeMinutes'] == df_utile['runtime'])]

In [26]:
# Création colonne RuntimeExact
df_a_garder_1['runtime_exact'] = df_a_garder_1['runtimeMinutes']

df_a_travailler_1 = df_utile[~(df_utile['runtimeMinutes'] == df_utile['runtime'])]

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_14904/3570380868.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_garder_1['runtime_exact'] = df_a_garder_1['runtimeMinutes']


In [27]:
# On met de côté là où une valeur manque seulement

df_a_garder_2 = df_a_travailler_1[((df_a_travailler_1['runtimeMinutes'].isna()) | (df_a_travailler_1['runtime'].isna())| (df_a_travailler_1['runtime'] == 0) | (df_a_travailler_1['runtimeMinutes'] == 0))]

In [28]:
# On conserve la donnée existante

df_a_garder_2['runtime_exact'] = df_a_garder_2[['runtimeMinutes', 'runtime']].max(axis = 1)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_14904/3521993277.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_garder_2['runtime_exact'] = df_a_garder_2[['runtimeMinutes', 'runtime']].max(axis = 1)


In [29]:
df_a_travailler_2 = df_a_travailler_1[~((df_a_travailler_1['runtimeMinutes'].isna()) | (df_a_travailler_1['runtime'].isna())| (df_a_travailler_1['runtime'] == 0) | (df_a_travailler_1['runtimeMinutes'] == 0))]
df_a_travailler_2['difference_runtime'] = abs(df_a_travailler_2['runtime'] - df_a_travailler_2['runtimeMinutes'])

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_14904/1261991755.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_travailler_2['difference_runtime'] = abs(df_a_travailler_2['runtime'] - df_a_travailler_2['runtimeMinutes'])


In [30]:
# On met de côté là où la différence de durée est négligeable

df_a_garder_3 = df_a_travailler_2[df_a_travailler_2['difference_runtime'] <= 20]

In [31]:
# On prend comme base la valeur max des 2 colonnes

df_a_garder_3['runtime_exact'] = df_a_garder_3[['runtimeMinutes', 'runtime']].max(axis = 1)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_14904/880983650.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_garder_3['runtime_exact'] = df_a_garder_3[['runtimeMinutes', 'runtime']].max(axis = 1)


In [32]:
df_a_travailler_3 = df_a_travailler_2[~(df_a_travailler_2['difference_runtime'] <= 20)]

In [33]:
# Pour le restant des données, on conserve la valeur max des 2 colonnes

df_a_travailler_3['runtime_exact'] = df_a_travailler_3[['runtimeMinutes', 'runtime']].max(axis = 1)
df_a_garder_4 = df_a_travailler_3

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_14904/264539281.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_travailler_3['runtime_exact'] = df_a_travailler_3[['runtimeMinutes', 'runtime']].max(axis = 1)


In [34]:
df_final = pd.concat([df_a_garder_1, df_a_garder_2, df_a_garder_3, df_a_garder_4])

In [35]:
df_a_travailler_1, df_a_travailler_2, df_a_travailler_3 = 0, 0, 0
df_a_garder_1, df_a_garder_2, df_a_garder_3, df_a_garder_4 = 0, 0, 0, 0

In [36]:
df_films = pd.concat([df_final, df_null])

In [37]:
df_final, df_null = 0, 0

VOTES

In [38]:
# On met de côté les valeurs inutiles

df_null5 = df_films[((df_films['vote_average'].isna()) & (df_films['averageRating'].isna()))]
df_null6 = df_films[((df_films['vote_average'] == 0) & (df_films['averageRating'] == 0))]
df_null7 = df_films[((df_films['vote_average'] == 0) & (df_films['averageRating'].isna()))]
df_null8 = df_films[((df_films['vote_average'].isna()) & (df_films['averageRating'] == 0))]

In [39]:
df_null9 = pd.concat([df_null5, df_null6, df_null7, df_null8])

In [40]:
df_null5, df_null6, df_null7, df_null8 = 0, 0, 0, 0

In [41]:
# On prend les valeurs utiles

df_ratings = df_films[~((df_films['vote_average'].isna()) & (df_films['averageRating'].isna()))]
df_ratings = df_ratings[~((df_ratings['vote_average'] == 0) & (df_ratings['averageRating'] == 0))]
df_ratings = df_ratings[~((df_ratings['vote_average'] == 0) & (df_ratings['averageRating'].isna()))]
df_ratings = df_ratings[~((df_ratings['vote_average'].isna()) & (df_ratings['averageRating'] == 0))]

In [42]:
df_ratings['vote_average'] = round(df_ratings['vote_average'], 1)

In [43]:
# On met de côté là où les 2 valeurs sont identiques

df_a_garder_1 = df_ratings[df_ratings['averageRating'] == df_ratings['vote_average']]
df_a_travailler_1 = df_ratings[~(df_ratings['averageRating'] == df_ratings['vote_average'])]

In [44]:
# On met de côté là où une valeur manque seulement

df_a_garder_2 = df_a_travailler_1[(df_a_travailler_1['averageRating'].isna()) | (df_a_travailler_1['vote_average'].isna())| (df_a_travailler_1['vote_average'] == 0) | (df_a_travailler_1['averageRating'] == 0)]
df_a_travailler_2 = df_a_travailler_1[~((df_a_travailler_1['averageRating'].isna()) | (df_a_travailler_1['vote_average'].isna())| (df_a_travailler_1['vote_average'] == 0) | (df_a_travailler_1['averageRating'] == 0))]

In [45]:
df_a_travailler_2['difference_vote'] = abs(df_a_travailler_2['averageRating'] - df_a_travailler_2['vote_average'])

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_14904/2546417221.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_travailler_2['difference_vote'] = abs(df_a_travailler_2['averageRating'] - df_a_travailler_2['vote_average'])


In [46]:
# On met de côté là où la différence de durée est négligeable

df_a_garder_3 = df_a_travailler_2[df_a_travailler_2['difference_vote'] <= 1]
df_a_travailler_3 = df_a_travailler_2[df_a_travailler_2['difference_vote'] > 1]

In [47]:
df_a_travailler_3['vote_exact'] = ((df_a_travailler_3['vote_count'] * df_a_travailler_3['vote_average']) + (df_a_travailler_3['numVotes'] * df_a_travailler_3['averageRating'])) / (df_a_travailler_3['numVotes'] + df_a_travailler_3['vote_count'])
df_a_travailler_3['vote_exact'] = round(df_a_travailler_3['vote_exact'], 1)
df_a_garder_4 = df_a_travailler_3

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_14904/4062215657.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_travailler_3['vote_exact'] = ((df_a_travailler_3['vote_count'] * df_a_travailler_3['vote_average']) + (df_a_travailler_3['numVotes'] * df_a_travailler_3['averageRating'])) / (df_a_travailler_3['numVotes'] + df_a_travailler_3['vote_count'])
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_14904/4062215657.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_travai

In [48]:
# On crée la colonne vote_exact pour garder la moyenne des votes

df_a_garder_1['vote_exact'] = df_a_garder_1[['averageRating', 'vote_average']].max(axis = 1)
df_a_garder_2['vote_exact'] = df_a_garder_2[['averageRating', 'vote_average']].max(axis = 1)
df_a_garder_3['vote_exact'] = df_a_garder_3[['averageRating', 'vote_average']].mean(axis = 1)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_14904/3653439724.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_garder_1['vote_exact'] = df_a_garder_1[['averageRating', 'vote_average']].max(axis = 1)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_14904/3653439724.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_garder_2['vote_exact'] = df_a_garder_2[['averageRating', 'vote_average']].max(axis = 1)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_14904/36534

In [49]:
# On concatène tous les tableaux conservés

df_final = pd.concat([df_a_garder_1, df_a_garder_2, df_a_garder_3, df_a_garder_4])
df_final['arrondi_vote_exact'] = round(df_final['vote_exact'], 0)

In [50]:
df_a_travailler_1, df_a_travailler_2, df_a_travailler_3 = 0, 0, 0
df_a_garder_1, df_a_garder_2, df_a_garder_3, df_a_garder_4 = 0, 0, 0, 0

In [51]:
df_films = pd.concat([df_final, df_null9])

In [52]:
df_final, df_null9 = 0, 0

GENRE

In [53]:
df_films = df_films.rename(columns = {'genres_x' : 'genres_y', 'genres_y' : 'genres_x'})

In [54]:
df_films[df_films[['genres_y', 'genres_x']] == '\\N'] = df_films[df_films[['genres_y', 'genres_x']] == '\\N'].replace('\\N', np.nan)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_14904/4290074865.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_films[df_films[['genres_y', 'genres_x']] == '\\N'] = df_films[df_films[['genres_y', 'genres_x']] == '\\N'].replace('\\N', np.nan)


In [55]:
def transfo_liste(x):
  if isinstance(x, str):
    return ast.literal_eval(x)
  else:
    return x
  
df_films['genres_y'] = df_films['genres_y'].apply(transfo_liste)

In [56]:
df_films['genres_y'] = df_films['genres_y'].apply(lambda x: x if (type(x) == list) else [])

In [57]:
df_films['genres_x'] = df_films['genres_x'].str.replace(",", ", ")

In [58]:
df_films['genres_x_list'] = df_films['genres_x'].fillna('').str.split(', ')

In [59]:
df_films['genre'] = df_films.apply(lambda row: list(set(row['genres_y'] + row['genres_x_list'])),axis=1)
# df_films = df_films.drop(['genres_x_list', 'genres_x', 'genres_y'], axis = 1)
df_films['genre'] = df_films['genre'].apply(lambda x: x if x != [] else np.nan)

In [60]:
df_films = df_films.drop(['genres_x_list', 'genres_x', 'genres_y'], axis = 1)

TITLE

In [61]:
df_films['title'] = df_films['original_title'].fillna(df_films['title_y']).fillna(df_films['title_x']).fillna(df_films['primaryTitle']).fillna(df_films['originalTitle'])
df_films = df_films.drop(['original_title', 'title_y', 'title_x', 'primaryTitle', 'originalTitle'], axis = 1)

In [62]:
df_films['adult'] = (df_films['isAdult'] == 1) | df_films['adult']
df_films = df_films.drop(['isAdult'], axis=1)

EXPORT

In [63]:
df_films.to_csv('P2_G5_films.csv.gz', index=False, compression='gzip')